# 1 - Présentation
## 1.1 - Introduction
Les réseaux sociaux sont une source de données presques infinis. Avec les Tweets sur Twitter ou les murs sur Facebook, il y a une multitude d'informations en attente d'être étudiées.

L'objectif de ce jeu de données est de prédire si un tweet est positif ou négatif. L'exercice nécessite de réaliser une analyse de sentiment en réalisant une classification à 2 sorties.

## 1.2 - Source de Données
Les données sont représentés par 2 dimensions :
* un chiffre de 0 ou 1 qui représente si un tweet est positif(1) ou négatif(0) ;
* une chaine de caractères représentant le tweet.

# 2 - Préparation des données
## 2.1 - Charger les librairies
Ci-dessous la liste des librairies nécessaires durant l'analyse des données :

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [ ]:
df = pd.read_csv('./data/training.csv', 
                 header=0,
                 sep=',',
                 names=['Sentimental','Id','Date','Query', 'Owner','Tweet'],
                 nrows = 1000,
                 encoding = 'utf8')

Maintenant nous allons afficher les 20 premières lignes de ce dataset pour mieux appréhender l'information :

In [ ]:
df.head(20)

Il  y a 6 dimensions dans ce dataset :
* Le sentiment du tweet (0 = negatif, 2 = neutre et 4 = positif) ;
* L'id du tweet ;
* La date de publication ;
* La requête. (S'il n'y en a aucune alors NO_QUERY) ;
* L'utilisateur qui a tweeté ;
* Le texte du tweet.

Nous allons afficher les types des données chargées :

In [ ]:
df.dtypes

Les types des données sont corrects pour l'analyse. Il n'est pas nécessaire de les casts.

## 2.3 - Nettoyer les données
Avant de passer à l'analyse nous allons nettoyer les données.

### 2.3.1 - Minuscule
Nous allons enlever les majuscules des différents tweets :

In [ ]:
df['Tweet'] = df['Tweet'].str.lower()

Cela va permettre de ne pas avoir de mots doublons pour la suite de l'analyse. Comme par exemple "awesome" et "Awesome" qui auraient été analysés comme des mots différents.

### 2.3.2 - @name

In [ ]:
df['Tweet'] = df['Tweet'].str.replace('@[\w]*', '')

### 2.3.3 - Ponctuation

In [ ]:
signs = ['.', ',' ,'!', '?', ';', "'", '-', '_', '&', '(', ')', '*']

for sign in signs:
    df['Tweet'] = df['Tweet'].str.replace(sign, '')

### 2.3.4 Strip

In [ ]:
df['Tweet'] = df['Tweet'].str.strip()

## 2.4 - Transformation
### 2.4.1 - Tokenize

In [ ]:
df['TweetTK'] = df.apply(lambda row: nltk.word_tokenize(row['Tweet']), axis=1)

In [ ]:
df.head(20)

### 2.4.2 - Stopword

In [ ]:
df['TweetTK'] = df['TweetTK'].apply(lambda x: [item for item in x if item not in stopwords.words('english')])
df['TweetTK'].head(20)

### 2.4.3 - Lemming

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
df['TweetTK'] = df['TweetTK'].apply(lambda x: [wordnet_lemmatizer.lemmatize(item) for item in x])

df['TweetTK'].head(20)

### 2.5 - Conclusion
# 3. Apprentissage

In [ ]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def find_features(document):
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features


tweets = []
for x, y in zip(df['TweetTK'], df['Sentimental']):
    tweets.append((x, y))


word_features = get_word_features(get_words_in_tweets(tweets))
word_features

In [ ]:
def extract_features(document):
    document_words = set(document)
    print(document)
    features = {}
    for word in tweet:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [ ]:
training_set = nltk.classify.apply_features(extract_features, tweets)

In [ ]:
print(training_set)